# Script XA DSA
This notebook imports necessary libraries and data from Excel files to work with ...

In [1]:
from xa_dose_analysis import dt_ids7_export_module as bh_utils
from xa_dose_analysis import mapping_module as bh_map
from xa_dose_analysis import plot_module as bh_plot



# Import the IDS7 data:
root_folder = "E:\\Intervensjon Doser\\IDS7\\2025"
df_ids7 = bh_utils.import_excel_files_to_dataframe(root_folder)

# Import the IDS7 data:
root_folder = "E:\\Intervensjon Doser\\DoseTrack - Serienivå\\2025"
df_dt = bh_utils.import_excel_files_to_dataframe(root_folder)

Reading E:\Intervensjon Doser\IDS7\2025\2025-01 - OUS.xlsx...
Reading E:\Intervensjon Doser\IDS7\2025\2025-07 - OUS.xlsx...
Reading E:\Intervensjon Doser\IDS7\2025\2025-05 - OUS.xlsx...
Reading E:\Intervensjon Doser\IDS7\2025\2025-04 - OUS.xlsx...
Reading E:\Intervensjon Doser\IDS7\2025\2025-09 - OUS.xlsx...
Reading E:\Intervensjon Doser\IDS7\2025\2025-10 - OUS.xlsx...
Reading E:\Intervensjon Doser\IDS7\2025\2025-02 - OUS.xlsx...
Reading E:\Intervensjon Doser\IDS7\2025\2025-11 - OUS.xlsx...
Reading E:\Intervensjon Doser\IDS7\2025\2025-06 - OUS.xlsx...
Reading E:\Intervensjon Doser\IDS7\2025\2025-12 - OUS.xlsx...
Reading E:\Intervensjon Doser\IDS7\2025\2025-03 - OUS.xlsx...
Reading E:\Intervensjon Doser\IDS7\2025\2025-08 - OUS.xlsx...
Reading E:\Intervensjon Doser\DoseTrack - Serienivå\2025\2025-01 - OUS.xlsx...
Reading E:\Intervensjon Doser\DoseTrack - Serienivå\2025\2025-07 - OUS.xlsx...
Reading E:\Intervensjon Doser\DoseTrack - Serienivå\2025\2025-05 - OUS.xlsx...
Reading E:\Interven

In [2]:
# Run all cleanups, filters and checks:
df_ids7 = bh_utils.run_all_cleanup_filters_and_checks(df_ids7, df_dt, True)
# Merge the dataframes:
data = bh_utils.merge_ids7_dt(df_ids7, df_dt, True)

Dropping unnecessary column: Prioritet- og lesemerkeikon
Dropping unnecessary column: Lagt til i demonstrasjon-ikon
Dropping unnecessary column: Status
Number of rows with NaT in the column "Bestilt dato og tidspunkt": 1588
Number of cancelled procedures: 85
Number of non-human subjects: 80
Number of rows with invalid accession number: 1
9295    NBOU000017196075
Name: Henvisnings-ID, dtype: str
Number of accession numbers in IDS7: 20665
Number of accession numbers in IDS7 not in DoseTrack: 5380
Without this column, we cannot keep track of which procedures are on the same patient.


User Guide to get the "Pasienter" column in the IDS7 data:


Note that the "Pasient column is not a column in IDS7, but must be created manually for anonymity reasons.
This can be done in excel after export to a safe location using the following method:
Copy the personal ID's from the column Fødelsnummer to a new column.
Then mark the new column and click the data fan -> "Fjern Duplikater"
Next to this colum

In [ ]:
# Print all unique elements in Modality Room, sorted alphabetically:
print("Unique elements in Modality Room:")
print(sorted(data['Modality Room'].unique()))

In [ ]:
# Representative doser for radiologisk Rikshospital
data_pci = data[data['Modality Room'].isin(['IRH_XA6', 'IRH_XA7', 'KRH_XA3', 'KRH_XA6', 'KRH_XA7', 'KRH_XA8','KUL_XA1', 'KUL_XA2', 'KUL_XA4'])]
import importlib
importlib.reload(pci_dict)

# Get the mapping dictionary:
mapping_dict = pci_dict.get_PCI_mapping_dict()
data_pci = bh_map.map_procedures(data_pci, mapping_dict, True)


In [ ]:
# Import all the exposure level data (This is going to be huge!):
# Import the IDS7 data:
root_folder = "E:\\Intervensjon Doser\\DoseTrack - Eksponeringsnivå\\2025"
df_dt_exp = bh_utils.import_excel_files_to_dataframe(root_folder)

In [ ]:
# Store the exposure level data as a pickle file for later use:
import pickle
with open('E:\\Intervensjon Doser\\DoseTrack - Eksponeringsnivå\\df_dt_exp_2025.pkl', 'wb') as f:
    pickle.dump(df_dt_exp, f)

In [3]:
# Open the pkl file and load the data:
import pickle
with open('E:\\Intervensjon Doser\\DoseTrack - Eksponeringsnivå\\df_dt_exp_2025.pkl', 'rb') as f:
    df_dt_exp = pickle.load(f)

In [ ]:
# Got throug all the entries in data_pci mapped as TAVI:
tavi_data = data_pci[data_pci['Mapped Procedures'] == 'TAVI']
# Get all the Accession Numbers for TAVI procedures:
tavi_accession_numbers = tavi_data['Accession Number'].unique()
# Filter the df_dt_exp dataframe to only include rows with Accession Numbers in tavi_accession_numbers:
tavi_exp_data = df_dt_exp[df_dt_exp['Accession Number'].isin(tavi_accession_numbers)]


In [ ]:
# Make a function to plot the amount of Air Kerma given at different angles for the procedure:
# The LAO/RAO angle is given by the column value: "Positioner Primary Angle (deg), increases from right to left (RAO to LAO)" 
# The Cranial/Caudal angle is given by the column value: "Positioner Secondary Angle (deg), increases from caudal to cranial"
# Each exposure has an Air Kerma value given by the column value: "Air Kerma (mGy)"
# The function divides the angles into bins of 10 degrees in a 2D grid, and sums the Air Kerma values for each bin. The function then plots a heatmap of the Air Kerma values for each bin of angles.
# The sum is then normalized to the total Air Kerma for the procedure, to get a percentage of the total dose given at each angle bin. The function then plots a heatmap of the percentage of the total dose given at each angle bin.
import importlib
from xa_dose_analysis import plot_module
importlib.reload(plot_module)
bh_plot = plot_module
bh_plot.plot_air_kerma_angle_heatmap(tavi_exp_data, 'TAVI', bin_size=10, save=False)



In [ ]:
def plot_deg(data, df_dt_exp, procedure_name, bin_size=20, save=False):
    import importlib
    from xa_dose_analysis import plot_module
    importlib.reload(plot_module)
    # Got throug all the entries in data_pci mapped as TAVI:
    procedure_data = data[data['Mapped Procedures'] == procedure_name]
    # Get all the Accession Numbers for the specific procedure:
    accession_numbers = procedure_data['Accession Number'].unique()
    # Filter the df_dt_exp dataframe to only include rows with Accession Numbers in tavi_accession_numbers:
    exp_data = df_dt_exp[df_dt_exp['Accession Number'].isin(accession_numbers)]
    bh_plot = plot_module
    bh_plot.plot_air_kerma_angle_heatmap(exp_data, procedure_name, bin_size, save=False)

In [ ]:
plot_deg(data_pci, df_dt_exp, 'TAVI', bin_size=10)

In [ ]:
plot_deg(data_pci, df_dt_exp, 'Mitraclip', bin_size=10)

In [ ]:
plot_deg(data_pci, df_dt_exp, 'Coronary angiography', bin_size=10)

In [ ]:
plot_deg(data_pci, df_dt_exp, 'PCI', bin_size=10)

In [ ]:
plot_deg(data_pci, df_dt_exp, 'PTSMA', bin_size=10)

In [ ]:
plot_deg(data_pci, df_dt_exp, 'PFO', bin_size=10)

In [ ]:
data_pacemaker = data[data['Modality Room'].isin(['KRH_Elfys1', 'KRH_Elfys1062', 'KRH_Elfys2', 'KRH_Elfys3','KRH_Elfys4', 'KRH_Elfys5', 'KRH_Lab13'])]
from xa_dose_analysis.mapping_dicts import mapping_dict_elfys_ecr as elfys_dict
import importlib
importlib.reload(elfys_dict)

# Get the mapping dictionary:
mapping_dict = elfys_dict.get_elfys_mapping_dict()
data_pacemaker = bh_map.map_procedures(data_pacemaker, mapping_dict, True)

In [ ]:
plot_deg(data_pacemaker, df_dt_exp, 'Ablation', bin_size=10)

In [ ]:
plot_deg(data_pacemaker, df_dt_exp, 'Electrophysiology', bin_size=10)

In [ ]:
plot_deg(data_pacemaker, df_dt_exp, 'Pacemaker', bin_size=10)

In [ ]:
carm_data = data_pacemaker[data_pacemaker['Modality Room'].isin(['KRH_Lab13'])]

In [ ]:
data_ped = data[data['Modality Room'].isin(['RRH_XA5'])]
# Below 18 years old:
data_ped = data_ped[data_ped['Age (Years)'] < 18]



In [ ]:
from xa_dose_analysis.mapping_dicts import mapping_dict_ped_card_ecr as ped_card_dict
import importlib
importlib.reload(ped_card_dict)

# Get the mapping dictionary:
mapping_dict = ped_card_dict.get_ped_card_mapping_dict()
data_ped = bh_map.map_procedures(data_ped, mapping_dict, True)

In [ ]:
plot_deg(data_ped, df_dt_exp, 'Pacemaker', bin_size=10)

In [ ]:
# print all modality rooms in data_pacemaker
print("Unique elements in Modality Rooms:")
print(sorted(data['Modality Room'].unique()))